In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

In [7]:
symbol = 's'
df = pd.read_csv(f'../datasets/{symbol}.csv')

In [8]:
y = df.trading_signal
X = df[['sma', 'macd', 'stochastic_k', 'stochastic_d', 'rsi', 'william_r']]
X = (X-X.min())/(X.max()-X.min())
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

In [9]:
model = SVR(C= 5, degree= 1, kernel= 'rbf')
model.fit(train_X, train_y)

predictions = model.predict(test_X)
print(f"MAE: {mean_absolute_error(test_y, predictions)}")

MAE: 0.21854632413975114


In [10]:
padding = np.full(train_X.shape[0], np.nan)
prediction_trading_signal = np.concatenate((padding, predictions), axis=None)
df['p_t_s_svm'] = prediction_trading_signal
df.to_csv(f'../datasets/{symbol}.csv')

In [ ]:
parameters = {
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': range(1,6),
    'C': range(1,6)   
}

grid = GridSearchCV(SVR(), parameters, cv = 2, n_jobs = 5, verbose=1)
grid.fit(train_X, train_y)

print(f"MAE: {mean_absolute_error(test_y, grid.predict(test_X))}")
print(grid.best_score_, grid.best_params_)